네이버 영화 TOP100 데이터 크롤링하기

라이브러리 설치

In [ ]:
#링크 수집을 위한 라이브러리 설치
!pip install selenium

     |████████████████████████████████| 911kB 7.0MB/s 


라이브러리 임포트

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

분석할 데이터 모으기(크롤링)

네이버 영화 TOP 100 페이지 별 링크 수집

In [ ]:
def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] # 100개의 영화링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo-1, lastPageNo) :
    # 실시간 TOP 100 영화 링크 수집
    url = 'https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_W&page=' + str(pageNo+1)

    req = requests.get(url) # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml')
    # html 파싱(원하는 위치에 있는 값을 찾아줌)을 위한 객체 생성, 불러올 데이터 정보가 lxml형식이다
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]')
    # 해당페이지의 페이지소스 중 378line 확인
    # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">
    # div 일종의 박스, 여기 안에 무슨 정보를 넣을 거다
    # ul(반복되는 정보) 안에 li가 들어있다(주로 그렇기 때문에 세트로 생각하면 좋다)

    # movielinks의 링크는 완전한 형태의 링크가 아니기 때문에 부족한 앞부분을 채워 완전한 링크를 만들어줘야함

    for movielink in movielinks :
      link = str(movielink.get('href'))

      links.append("https://series.naver.com" + link) # 접속가능한 전체링크 형태로 변환하여 links 리스트에 넣어줌
      
  return links

네이버 영화 제목,평점,장르,줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links):
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = "https://www.naver.com"

  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url2)
  time.sleep(3.0) # 3초 sleep

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't')

  for link in links:
    print(link + '수집 중.........')
    driver.switch_to_window(driver.window_handles[-1]) 
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to_window(driver.window_handles[0]) # 새창 활성화
    time.sleep(3.0)
    
    html_source = driver.page_source # req.text 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년관람불가 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag =''.join(flag)
    newflag = newflag.replace('\n','')

    if newflag =='네이버':
      time.sleep(1.0)

      #평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score) # 정수형으로 변경
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르','')
      genre = genre.replace('\n','')
      genre = genre.replace('\t','')
      genre_infos.append(genre)

      # 제목, 줄거리
      
      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      #영화 상세보기 페이지 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta", {"property":"og:title"}).get('content')
      title_infos.append(title)

      #줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0:
        content_infos.append("줄거리 오류")

      else:
        for contents in contents_texts :
          # 줄거리 데이터 클랜징 작업
          temp = contents.text
          temp = temp.replace("\r",'')  # 줄바꿈 제거
          temp = temp.replace("\xa0",'') #공백 제거
          content_infos.append(temp)
    
    elif newflag =='':
      print('청불 영화로 데이터 수집하지 않습니다.')
  
  print('수집 완료 합니다.....')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {"제목": title_infos, "평점":score_infos, "장르":genre_infos, "줄거리":content_infos}

  #딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

  #수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates("줄거리", keep='first') #줄거리 비교해서 중복된 영화를 삭제(첫번째 수집 영화만 남김)

  return movie_df2


CSV 파일로 저장

In [ ]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(('movie_my_data_' + str(num) + '.csv' ),
                    sep=',', na_rep='NaN', encoding='euc-kr')
    
  except:
    print('Error')  

크롤링 실행

In [ ]:
#1) 크롤링 할 링크 수집
links = getPageLinksWantRange(1,5)

#2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

#3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

https://series.naver.com/movie/detail.nhn?productNo=6291892수집 중.........


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use driver.switch_to.window instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: use driver.switch_to.window instead


9
https://series.naver.com/movie/detail.nhn?productNo=3400212수집 중.........
7
https://series.naver.com/movie/detail.nhn?productNo=2822635수집 중.........
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3024534수집 중.........
8
https://series.naver.com/movie/detail.nhn?productNo=6283368수집 중.........
6
https://series.naver.com/movie/detail.nhn?productNo=6023769수집 중.........
9
https://series.naver.com/movie/detail.nhn?productNo=3400211수집 중.........
7
https://series.naver.com/movie/detail.nhn?productNo=3639460수집 중.........
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6243813수집 중.........
9
https://series.naver.com/movie/detail.nhn?productNo=6119777수집 중.........
7
https://series.naver.com/movie/detail.nhn?productNo=6139375수집 중.........
9
https://series.naver.com/movie/detail.nhn?productNo=6257268수집 중.........
9
https://series.naver.com/movie/detail.nhn?productNo=6253746수집 중.........
8
https://series.naver.com/movie/detail.nhn?productNo=615887

#크롤링한 TOP 100 영화 데이터 랭킹 시각화

라이브러리 임포트

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

데이터 준비하기

In [ ]:
df = pd.read_csv('/content/movie_my_data_weekly.csv', encoding='euc-kr')
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  63 non-null     int64 
 1   제목          63 non-null     object
 2   평점          63 non-null     int64 
 3   장르          63 non-null     object
 4   줄거리         63 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB


In [ ]:
df.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,파이프라인,6,액션,손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [ ]:
fig = px.scatter(df, x='제목', y='평점', color='장르')
fig.show()

In [ ]:
df_score = df[df.평점 ==9]
df_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
58,73,태극기 휘날리며,9,액션,가족의 생계를 책임지기 위해 열심히 살아가는 ‘진태’(장동건)는 약혼녀 ‘영신’(이...
59,74,소년시절의 너,9,액션,시험만 잘 치면 멋진 인생을 살 수 있다고 가르치는 세상에서기댈 곳 없이 세상에 내...
60,76,하울의 움직이는 성,9,애니메이션,"어느 날, 영문도 모른 채 마녀의 저주로 인해 할머니가 된 소녀 '소피'절망 속에서..."
61,77,101마리의 달마시안 개,9,액션,악녀 크루엘라 드빌의 하수인에 달마시안 강아지들이 납치되고 강아지들의 부모인 퐁고와...
62,78,나의 히어로 아카데미아 더 무비: 두 명의 히어로,9,애니메이션,"모두가 ‘개성’이라는 초능력을 가진 세계, 여기에 ‘무개성’의 소년이 있었다!하지만..."


9점 평점의 영화 중 가장 큰 비율을 차지하고 있는 장르는?

In [ ]:
fig  = px.pie(df_score, values='평점', names='장르')
fig.show()